In [1]:
from extractors.pdf_extractor import extract_pdf_as_markdown,extract_docx_as_markdown
from chains.audit_chain import analyze_module
from chains.evidence_chain import analyze_evidence
from chains.improvement_chain import get_improvement_chain
from langchain_community.chat_models import ChatOpenAI
import pandas as pd
from dotenv import load_dotenv
import asyncio
import re
import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

In [2]:
load_dotenv()
# print(os.getenv("OPENAI_API_KEY"))

file_path = "./files/"
controls = pd.read_json("ISO_27001_2022_Controls_List.json")


In [3]:
controls

,Section,Control ID,Control Title,Clause
0,A.5 (Organisational Controls),A.5.1,Policies for information security,"4,5"
1,A.5 (Organisational Controls),A.5.2,Information security roles and responsibilities,"4,5"
2,A.5 (Organisational Controls),A.5.3,Segregation of duties,5
3,A.5 (Organisational Controls),A.5.4,Management responsibilities,"5,6,9,10"
4,A.5 (Organisational Controls),A.5.5,Contact with authorities,
...,...,...,...,...
88,A.8 (Technological Controls),A.8.30,Outsourced development,
89,A.8 (Technological Controls),A.8.31,"Separation of development, test and production...",6
90,A.8 (Technological Controls),A.8.32,Change management,8
91,A.8 (Technological Controls),A.8.33,Test information,


In [4]:
import numpy as np

n = len(controls)
part_size = n // 5
remainder = n % 5

# Calculate split indices
sizes = [part_size + (1 if i < remainder else 0) for i in range(5)]
indices = np.cumsum([0] + sizes)

dfs = [controls.iloc[indices[i]:indices[i+1]].reset_index(drop=True) for i in range(5)]
df1, df2, df3, df4, df5 = dfs

In [5]:
def extract_file_as_markdown(file_path):
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".pdf":
        return extract_pdf_as_markdown(file_path)
    elif ext == ".docx":
        return extract_docx_as_markdown(file_path)
    else:
        raise ValueError(f"Unsupported file type: {ext}")

In [6]:
def read_folder_and_join_markdown(folder_path, file_list=None):
    """
    Args:
        folder_path (str): Root folder to search.
        file_list (list[str], optional): List of filenames (with or without relative paths) to include.
                                         If None, include all .pdf/.docx files.

    Returns:
        str: Combined markdown string from all valid files.
    """
    all_markdown = []
    normalized_file_list = set(os.path.normpath(f).lower() for f in file_list) if file_list else None
    print(f"Normalized file list: {normalized_file_list}")

    for root, _, files in os.walk(folder_path):
        for filename in files:
            ext = os.path.splitext(filename)[1].lower()
            if ext not in [".pdf", ".docx"]:
                continue

            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, folder_path)
            normalized_relative_path = os.path.normpath(relative_path).lower()

            if normalized_file_list and normalized_relative_path not in normalized_file_list:
                continue  # Skip files not in the list

            try:
                markdown = extract_file_as_markdown(file_path)
                all_markdown.append(f"## File: {relative_path}\n\n{markdown}")
            except Exception as e:
                print(f"❌ Error processing {file_path}: {e}")

    return "\n\n---\n\n".join(all_markdown)
def read_folder_and_join_markdown_exclude(folder_path, exclude_file_list=None):
    """
    Args:
        folder_path (str): Root folder to search.
        exclude_file_list (list[str], optional): List of file paths (relative to folder_path) to exclude.

    Returns:
        str: Combined markdown string from all valid, non-excluded files.
    """
    all_markdown = []
    normalized_exclude_list = set(os.path.normpath(f).lower() for f in exclude_file_list) if exclude_file_list else set()

    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.startswith("._"):
                continue  
            ext = os.path.splitext(filename)[1].lower()
            if ext not in [".pdf", ".docx"]:
                continue

            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, folder_path)
            normalized_relative_path = os.path.normpath(relative_path).lower()

            if normalized_relative_path in normalized_exclude_list:
                continue  # Skip files in the exclusion list

            try:
                markdown = extract_file_as_markdown(file_path)
                all_markdown.append(f"## File: {relative_path}\n\n{markdown}")
            except Exception as e:
                print(f"❌ Error processing {file_path}: {e}")

    return "\n\n---\n\n".join(all_markdown)

In [7]:
with open("output.md", 'r', encoding='utf-8') as f:
    evidence_text = f.read()
with open("policy_text.md", 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
file_list = ["Business Continuity Plan E-WISE [Restricted].docx.pdf","Information Security Policy [Internal].docx","Initial Implementation Plan - AI-Driven Document Compliance Analysis System (3).pdf"]
# file_list = ["Business Continuity Plan E-WISE [Restricted].docx.pdf","Information Security Policy [Internal].docx"]
evidence_text = read_folder_and_join_markdown_exclude(file_path, exclude_file_list=file_list)
text = read_folder_and_join_markdown(file_path,file_list=file_list)


✔️ Tesseract detected: tesseract 4.1.1
❌ Error processing ./files/Evidence controls are applied/Pentest_Reducate_Report_EN (1).pdf: cannot write mode RGBA as JPEG
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detec

In [8]:
len(evidence_text)

43753

In [9]:

# llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3, api_key=os.getenv("OPENAI_API_KEY"))
# clauses = [df1,df2,df3,df4,df5]
# audits = []
# results = []
# for clause in clauses:
#     audit_chain = analyze_module(llm)
#     results.append(audit_chain.run(text=text, control_json=clause.to_json(orient='records', indent=2)))



In [10]:
from langchain.callbacks import get_openai_callback
import nest_asyncio
import sys
nest_asyncio.apply()

# text = extract_pdf_as_markdown(file_path)

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))
clauses = [df1,df2,df3,df4,df5]
results = []
total_tokens = 0
total_cost = 0.0

/tmp/ipykernel_17412/2118181688.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))


In [ ]:


async def process_clause_async(clause, text, llm):
    def sync_callback_wrapper():
        with get_openai_callback() as cb:
            audit_chain = analyze_module(llm)
            result = audit_chain.run(
                text=text,
                control_json=clause.to_json(orient='records', indent=2)
            )
            return {
                "result": result,
                "tokens": cb.total_tokens,
                "cost": cb.total_cost,
                "controls": len(clause)
            }

    return await asyncio.to_thread(sync_callback_wrapper)

async def run_all_clauses():
    results = []
    total_tokens = 0
    total_cost = 0.0

    tasks = [process_clause_async(clause, text, llm) for clause in clauses]
    clause_results = await asyncio.gather(*tasks)

    for data in clause_results:
        results.append(data["result"])
        total_tokens += data["tokens"]
        total_cost += data["cost"]
        print(f"🔎 Clause Processed: {data['controls']} controls")
        print(f"🧠 Tokens used: {data['tokens']}")
        print(f"💵 Cost: ${data['cost']:.6f}")

    print("\n✅ All clauses processed (async).")
    print(f"🔢 Total tokens used: {total_tokens}")
    print(f"💰 Total cost: ${total_cost:.6f}")

    return results

if __name__ == "__main__":
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    final_results = await run_all_clauses() if 'google.colab' in sys.modules or 'IPython' in sys.modules else asyncio.run(run_all_clauses())
    


/tmp/ipykernel_9857/2935787740.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))
/tmp/ipykernel_9857/2935787740.py:18: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = audit_chain.run(
/home/sherry/DocAnalyzer/Document_Analyzer_Streamlit_App/chains/audit_chain.py:51: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm, prompt=prompt)

🔎 Clause Processed: 19 controls
🧠 Tokens used: 32244
💵 Cost: $0.016348
🔎 Clause Processed: 19 controls
🧠 Tokens used: 32151
💵 Cost: $0.016152
🔎 Clause Processed: 19 controls
🧠 Tokens used: 32119
💵 Cost: $0.016193
🔎 Clause Processed: 18 controls
🧠 Tokens used: 32557
💵 Cost: $0.016952
🔎 Clause Processed: 18 controls
🧠 Tokens used: 32114
💵 Cost: $0.016222

✅ All clauses processed (async).
🔢 Total tokens used: 161185
💰 Total cost: $0.081867


In [40]:
final_results = pd.read_json("policy.json")

In [42]:
import json
json_obj = final_results[["Control Id", "Clause", "Control Title", "Policy"]].to_dict(orient='records')

# Step 2: Serialize to JSON string
final_results = json.dumps(json_obj, indent=2)

In [43]:
evidence_chain = analyze_evidence(llm)
result = evidence_chain.run(
    text=evidence_text,
    control_json=final_results
)

In [44]:
result

'[\n  {\n    "Control Id": "A.5.1",\n    "Policy Implementation Evidence": "Managment Review Team Meeting E-WISE 2025 [Restricted].docx.pdf",\n    "Exact Evidence Extract": "Information Security Policy is referenced as an internal document relevant to the management review meeting (Page 16). The policy is shared with all employees and information security goals are communicated and added to training (Page 14).",\n    "Evidence Recommendation": "Include explicit evidence of the policy approval by the director and documented yearly review by ISSC and ISWG in the evidence.",\n    "Evidence": "🟡"\n  },\n  {\n    "Control Id": "A.5.2",\n    "Policy Implementation Evidence": "Managment Review Team Meeting E-WISE 2025 [Restricted].docx.pdf",\n    "Exact Evidence Extract": "Roles such as Managing Director, System Administrator, Project Manager/ISMS Responsible, and others are listed as attendees in management review meetings (Page 2). Responsibilities for policy enforcement, training, and revi

In [45]:
import json
results=result
with open("evidence_report.json", "w") as f:
    f.write(json.dumps(results, indent=2, ensure_ascii=False))

In [46]:
results

'[\n  {\n    "Control Id": "A.5.1",\n    "Policy Implementation Evidence": "Managment Review Team Meeting E-WISE 2025 [Restricted].docx.pdf",\n    "Exact Evidence Extract": "Information Security Policy is referenced as an internal document relevant to the management review meeting (Page 16). The policy is shared with all employees and information security goals are communicated and added to training (Page 14).",\n    "Evidence Recommendation": "Include explicit evidence of the policy approval by the director and documented yearly review by ISSC and ISWG in the evidence.",\n    "Evidence": "🟡"\n  },\n  {\n    "Control Id": "A.5.2",\n    "Policy Implementation Evidence": "Managment Review Team Meeting E-WISE 2025 [Restricted].docx.pdf",\n    "Exact Evidence Extract": "Roles such as Managing Director, System Administrator, Project Manager/ISMS Responsible, and others are listed as attendees in management review meetings (Page 2). Responsibilities for policy enforcement, training, and revi

In [47]:
final_results

'[\n  {\n    "Control Id": "A.5.1",\n    "Clause": "4,5",\n    "Control Title": "Policies for information security",\n    "Policy": "The Information Security Policy document defines baseline control measures, scope, enforcement, and responsibilities. It is approved by the director and reviewed yearly by ISSC and ISWG. It covers confidentiality, compliance expectations, and management oversight."\n  },\n  {\n    "Control Id": "A.5.2",\n    "Clause": "4,5",\n    "Control Title": "Information security roles and responsibilities",\n    "Policy": "Roles and responsibilities are clearly defined for CEO, ISSC, ISWG, operational directors, IT asset managers, managers, users, and executive responsibilities. Responsibilities include resource allocation, policy enforcement, awareness, and review."\n  },\n  {\n    "Control Id": "A.5.3",\n    "Clause": "5",\n    "Control Title": "Segregation of duties",\n    "Policy": "Segregation of duties is mandated to reduce risk of misuse. Where segregation is

In [48]:
def load_json_report(results):
    # Step 1: Extract content from wrapper
    if isinstance(results, list):
        if len(results) == 1 and isinstance(results[0], str):
            raw_text = results[0]
        elif all(isinstance(item, dict) for item in results):
            return results
        else:
            raw_text = ''.join(results)
    elif isinstance(results, str):
        raw_text = results
    else:
        raw_text = str(results)
 
    # Step 2: Remove markdown fences
    raw_text = raw_text.strip()
    if raw_text.startswith("```json"):
        raw_text = raw_text.removeprefix("```json").strip()
    if raw_text.endswith("```"):
        raw_text = raw_text.removesuffix("```").strip()
 
    # Step 3: Use regex to extract all JSON arrays
    try:
        # Extract all top-level lists using regex
        arrays = re.findall(r'\[\s*{.*?}\s*\]', raw_text, re.DOTALL)
        combined = []
        for arr in arrays:
            data = json.loads(arr)
            if isinstance(data, list):
                combined.extend(data)
            else:
                combined.append(data)
        return combined
 
    except json.JSONDecodeError as e:
        print("❌ JSON parsing failed:", e)
        print("--- JSON Preview ---")
        print(raw_text[:500])
        raise

In [49]:
# Step 3: Convert to DataFrame
parsed_results = load_json_report(results)
parsed_final_results = load_json_report(final_results)
resutls_df = pd.DataFrame(parsed_results)
final_result_df= pd.DataFrame(parsed_final_results)

In [50]:
resutls_df

,Control Id,Policy Implementation Evidence,Exact Evidence Extract,Evidence Recommendation,Evidence
0,A.5.1,Managment Review Team Meeting E-WISE 2025 [Res...,Information Security Policy is referenced as a...,Include explicit evidence of the policy approv...,🟡
1,A.5.2,Managment Review Team Meeting E-WISE 2025 [Res...,"Roles such as Managing Director, System Admini...",Provide documented role descriptions or a form...,🟡
2,A.5.3,Managment Review Team Meeting E-WISE 2025 [Res...,Segregation of duties is addressed in audit re...,Include documented compensating controls or mo...,🟡
3,A.5.4,Managment Review Team Meeting E-WISE 2025 [Res...,Management review meetings cover setting secur...,None,✅
4,A.5.5,No direct evidence found in provided documents.,No documented procedures or contact details fo...,Document and provide evidence of procedures an...,❌
5,A.5.6,Managment Review Team Meeting E-WISE 2025 [Res...,Subscriptions to external security news and vu...,Explicitly document subscriptions and frequenc...,🟡
6,A.5.7,Vendor risk assesment - Jumpcloud [Internal].d...,Vendor risk assessment includes evaluation of ...,None,✅
7,A.5.8,A.5.8/4.4.1.6.1 System development life cycle ...,SDLC process documented with testing stages in...,None,✅
8,A.5.9,No direct evidence found in provided documents.,No explicit inventory document or evidence of ...,Maintain and provide evidence of IT asset inve...,❌
9,A.5.10,Managment Review Team Meeting E-WISE 2025 [Res...,Centralised password manager 'Zoho Vault' impl...,Include explicit acceptable use policy documen...,🟡


In [51]:
final_result_df

,Control Id,Clause,Control Title,Policy
0,A.5.1,"4,5",Policies for information security,The Information Security Policy document defin...
1,A.5.2,"4,5",Information security roles and responsibilities,Roles and responsibilities are clearly defined...
2,A.5.3,5,Segregation of duties,Segregation of duties is mandated to reduce ri...
3,A.5.4,"5,6,9,10",Management responsibilities,Management is responsible for setting security...
4,A.5.5,,Contact with authorities,Procedures exist for contacting local authorit...
...,...,...,...,...
87,A.8.30,,Outsourced development,Outsourced development requires review of SDLC...
88,A.8.31,6,"Separation of development, test and production...","Development, test, and production environments..."
89,A.8.32,8,Change management,Changes to information processing facilities a...
90,A.8.33,,Test information,"Test environments use limited, anonymised, or ..."


In [52]:
final_result_df.to_json("policy.json")

In [53]:
parsed_final_results

[{'Control Id': 'A.5.1',
  'Clause': '4,5',
  'Control Title': 'Policies for information security',
  'Policy': 'The Information Security Policy document defines baseline control measures, scope, enforcement, and responsibilities. It is approved by the director and reviewed yearly by ISSC and ISWG. It covers confidentiality, compliance expectations, and management oversight.'},
 {'Control Id': 'A.5.2',
  'Clause': '4,5',
  'Control Title': 'Information security roles and responsibilities',
  'Policy': 'Roles and responsibilities are clearly defined for CEO, ISSC, ISWG, operational directors, IT asset managers, managers, users, and executive responsibilities. Responsibilities include resource allocation, policy enforcement, awareness, and review.'},
 {'Control Id': 'A.5.3',
  'Clause': '5',
  'Control Title': 'Segregation of duties',
  'Policy': 'Segregation of duties is mandated to reduce risk of misuse. Where segregation is difficult, compensating controls like monitoring and audit tr

In [54]:
merged_df = pd.merge(final_result_df, resutls_df, on="Control Id", how="outer")


In [55]:
merged_df['Evidence'] = merged_df['Evidence'].replace('', pd.NA).fillna('❌')

In [56]:
merged_df

,Control Id,Clause,Control Title,Policy,Policy Implementation Evidence,Exact Evidence Extract,Evidence Recommendation,Evidence
0,A.5.1,"4,5",Policies for information security,The Information Security Policy document defin...,Managment Review Team Meeting E-WISE 2025 [Res...,Information Security Policy is referenced as a...,Include explicit evidence of the policy approv...,🟡
1,A.5.10,6,Acceptable use of information and other associ...,Acceptable use policies are defined covering I...,Managment Review Team Meeting E-WISE 2025 [Res...,Centralised password manager 'Zoho Vault' impl...,Include explicit acceptable use policy documen...,🟡
2,A.5.11,,Return of assets,Procedures require return of company assets (l...,A.5.25/#18- CAPA-form [Confidential] - possibl...,Incident involving data leakage during employe...,None,✅
3,A.5.12,6,Classification of information,Information classification schemes are establi...,No direct evidence found in provided documents.,No explicit information classification scheme ...,Document and provide evidence of information c...,❌
4,A.5.13,,Labelling of information,Information must be labelled and handled accor...,No direct evidence found in provided documents.,No explicit labelling guidelines or evidence f...,Provide documented labelling guidelines and ev...,❌
...,...,...,...,...,...,...,...,...
87,A.8.5,,Secure authentication,"Section 4.1 mandates password confidentiality,...",NaN,NaN,NaN,❌
88,A.8.6,8,Capacity management,Section 3.3.1 Business Partnership Contracts i...,NaN,NaN,NaN,❌
89,A.8.7,8,Protection against malware,Section 6.1 and 6.6 detail malware protection ...,NaN,NaN,NaN,❌
90,A.8.8,8,Management of technical vulnerabilities,Section 6.2 defines a formal patch and vulnera...,NaN,NaN,NaN,❌


In [57]:
merged_df.to_excel("result.xlsx")

In [58]:
with open("output.md", "w", encoding="utf-8") as f:
    f.write(evidence_text)

print("✅ PDF content extracted and saved as 'output.md'")

✅ PDF content extracted and saved as 'output.md'


In [59]:
with open("policy_text.md", "w", encoding="utf-8") as f:
    f.write(text)

print("✅ PDF content extracted and saved as 'output.md'")

✅ PDF content extracted and saved as 'output.md'
